# Импортируем необходимые библиотеки

In [13]:
import json
from datetime import datetime
from datetime import timedelta
import time
import requests
import re
import pandas as pd
import numpy as np
import os
import yaml
import docker

# Опередляем объём импорта и необходимые объекты импорта

In [9]:
import sql_handler as sql
import yaml
has_errors = False
table_settings_mapper = {
    'deal': 'deal_fields',
    'contact': 'contact_fields',
    'company': 'company_fields',
    'lead': 'lead_fields',
}
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
if config['database_type'] == 'MySQL':
    columns = sql.get_columns_and_types_sql(config['table_names']['deal'], **config['db'])
    set_of_columns = set([column['name'] for column in columns])
    for column in config['deal_fields']:
        if config['deal_fields'][column] not in set_of_columns:
            if has_errors == False:
                has_errors = True
                list_of_errors = []
            list_of_errors.append(f"Column {column} not found in table {config['table_names']['deal']}")
if has_errors:
    print(*list_of_errors, '\n')
    print('Please, check your config.yaml file and database. After that, restart the container.')
    client = docker.from_env()
    container = client.containers.get('bitrix24_loader')
    container.stop()


Column bla_bla not found in table deals_b24 



In [10]:
import sql_handler as sql
import yaml
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
sql.test_connection(**config['db'])

Successfully connected to the database.
